# Tutorial: Cognitive Service Vision Image Composition using Python

This is a tutorial about using Cognitive Service Vision Image Composition. Cognitive Service Vision Image Composition are collection of tools to pre-process images before running product recognition, so that better result can be achieved

Currently, image composition feature are available in **EastUS**, **West US2**, and **West Europe** regions.

Please create a computer vision resource on Azure portal, in **EastUS**, **West US2**, or **West Europe** region, if you don't already have one. You can use [Multi-service resource](https://learn.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Canomaly-detector%2Clanguage-service%2Ccomputer-vision%2Cwindows) as well. 

## Install the python samples package

Install the sample code including utility code helping you use Python to run planogram compliance in this tutorial.

In [ ]:
pip install cognitive-service-vision-product-recogntion-python-samples

## Credentials

Resource name and resource key are needed for accessing the service, which you can find here:

![check media/credentials.png if pic does not show up](./resources/credentials.png)

In [ ]:
# Resource and key
import logging
logging.getLogger().setLevel(logging.INFO)
from cognitive_service_vision_model_customization_python_samples import ResourceType

resource_type = ResourceType.SINGLE_SERVICE_RESOURCE # or ResourceType.MULTI_SERVICE_RESOURCE

resource_name = None
multi_service_endpoint = None

if resource_type == ResourceType.SINGLE_SERVICE_RESOURCE:
    resource_name = '{specify_your_resource_name}'
    assert resource_name
else:
    multi_service_endpoint = '{specify_your_service_endpoint}'
    assert multi_service_endpoint

resource_key = '{specify_your_resource_key}'

## Prepare and load the planogram

What's a planogram

Sample planogram

## Call product recognition

## Match product recognition result with planogram to check for compliance